In [1]:
import numpy as np
from fFunc import Ffunc
from gFunc import Gfunc
from astropy.constants import G, M_sun
from scipy.integrate import quad
import astropy.units as u
from scipy.optimize import curve_fit

In [7]:
#All distances are in kpc, all masses in Msun units. 
class ModelVelocity(object):

    def __init__(self, n, Mbulge, re_bulge, Mbh, sigma_B, r_in, r_out, g_func, f_func=None):

        #Save the input parameters.
        self.n = n
        self.Mbulge = Mbulge
        self.re_bulge = re_bulge
        self.Mbh = Mbh
        self.sigma_B = sigma_B
        self.r_in = r_in
        self.r_out = r_out
        self.g_func = g_func

        #Start the the f and g function objects.
        if f_func is None:
            self.f_func = Ffunc(sigma_B, r_in, r_out)
        else:
            self.f_func = f_func

        #Get the convenience constant Ie
        g_func_rnorm_max = g_func.rnorm_max
        self.Ie_bulge = Mbulge/g_func.g_interp((n,g_func_rnorm_max))

        #Save the bn value for n=1/2 (gaussian), which we need for the light profile. 
        self.b_gaussian = g_func.b_n(0.5)

        #Constant with the right units for getting sigma. 
        self.K = (((2./3.) * G * M_sun/(1.*u.kpc))**0.5).to(u.km/u.s).value

        return
    
    def sigma(self, r):

        rnorm = r/self.re_bulge
        host_mass = self.Ie_bulge * self.g_func.g_interp((self.n, rnorm))
        return self.K * ((self.Mbh+host_mass)/r)**0.5

    def Iv(self, v):
    
        func = lambda x: np.exp(-self.b_gaussian*x**2-0.5*(v/self.sigma(x))**2) * self.f_func.f_interp(x)*x

        return quad(func, 0., 5., epsrel=1e-3)[0]
    
    def Iv_nosmearing(self, v):
    
        func = lambda x: np.exp(-self.b_gaussian*x**2-0.5*(v/self.sigma(x))**2)

        return quad(func, self.r_in, self.r_out)[0]


In [3]:
g_func = Gfunc()
n = 4
Mbulge = 5e10
re_bulge = 0.8
Mbh = 1e10
sigma_B = 0.5
r_in = 0.
r_out = 0.5
modelv = ModelVelocity(n, Mbulge, re_bulge, Mbh, sigma_B, r_in, r_out, g_func)

In [4]:
vs = np.arange(-2000, 2000, 100)
Ivs = np.zeros(vs.shape)
for i, v in enumerate(vs):
    Ivs[i] = modelv.Iv(v)


In [ ]:
Ivs_nsm = np.zeros(vs.shape)
for i, v in enumerate(vs):
    Ivs_nsm[i] = modelv.Iv_nosmearing(v)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(vs, Ivs)
plt.plot(vs, Ivs_nsm * np.max(Ivs)/np.max(Ivs_nsm))

In [3]:
def gauss(x, *p):
    A, sigma = p
    return A*np.exp(-x**2/(2.*sigma**2))

In [ ]:
p0 = [0.1, 300.]
coeff, var_matrix = curve_fit(gauss, vs, Ivs, p0=p0)
print(coeff[1])
coeff, var_matrix = curve_fit(gauss, vs, Ivs_nsm, p0=p0)
print(coeff[1])

In [ ]:
vs = np.arange(0, 2000, 100)
Ivs = np.zeros(vs.shape)
for i, v in enumerate(vs):
    Ivs[i] = modelv.Iv(v)
p0 = [0.1, 300.]
coeff, var_matrix = curve_fit(gauss, vs, Ivs, p0=p0)
print(coeff[1])

In [ ]:
Ivs_nsm = np.zeros(vs.shape)
for i, v in enumerate(vs):
    Ivs_nsm[i] = modelv.Iv_nosmearing(v)

In [ ]:
p0 = [0.1, 300.]
coeff, var_matrix = curve_fit(gauss, vs, Ivs, p0=p0)
print(coeff[1])
coeff, var_matrix = curve_fit(gauss, vs, Ivs_nsm, p0=p0)
print(coeff[1])

In [4]:
g_func = Gfunc()
n = 4
Mbulge = 5e10
re_bulge = 0.8
Mbh = 1e10

In [11]:
sigma_B = 0.5
r_ins = [0., 0.5, 1.0, 1.5]
r_outs = [0.5, 1.0, 1.5, 2.0]
f_funcs = [None]*len(r_outs)
for i in range(len(r_ins)):
    f_funcs[i] = Ffunc(sigma_B, r_ins[i], r_outs[i])

In [14]:
for i in range(len(r_ins)):
    r_in = r_ins[i]
    r_out = r_outs[i]
    modelv = ModelVelocity(n, Mbulge, re_bulge, Mbh, sigma_B, r_in, r_out, g_func, f_func=f_funcs[i])

    vs = np.arange(0, 20000, 100)
    Ivs = np.zeros(vs.shape)
    for i, v in enumerate(vs):
        Ivs[i] = modelv.Iv(v)
    p0 = [0.1, 300.]
    coeff, var_matrix = curve_fit(gauss, vs, Ivs, p0=p0)
    print(coeff[1])

/var/folders/p7/drxzchtj4yb641v79lt0tjyh0000gn/T/ipykernel_81871/1972671258.py:44: IntegrationWarning: The integral is probably divergent, or slowly convergent.
  return quad(func, 0., 5., epsrel=1e-3)[0]


396.91934691770973
363.10576484362326
330.5938364444454
303.0216462782406
